In [2]:
import json 
from pat_sql import PatJob
from common import AppSettings

# Submit Job

In [3]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

{'job_guid': '80a1ca07-41f9-4f34-b7bb-d01b477bc78d',
 'job_name': 'PAT_Test_AIG',
 'error_action': 'continue',
 'type_of_rating': 'PSOLD',
 'coverage': 'Building + Contents + Time Element',
 'peril_subline': 'All_Perils',
 'subject_premium': 100000000.0,
 'loss_alae_ratio': 1,
 'average_accident_date': '1/1/2022',
 'trend_factor': 1.035,
 'additional_coverage': 2,
 'deductible_treatment': 'Retains Limit',
 'server': 'DFWCAT-RMS5SQL1',
 'edm_database': 'RMS_EPL_AIG_PAT_Testing_202107_EDM181',
 'rdm_database': 'RMS_EPL_AIG_PAT_Testing_202107_RDM181',
 'portinfoid': 6,
 'perilid': 5,
 'analysisid': 5,
 'user_name': 'cxiao',
 'user_email': 'chengyou.xiao@guycarp.com'}

# Run Sequential Code 

In [ ]:
%%time

job = PatAnalysis(job_para) 

job.extract_edm_rdm()
if job.check_data():
    if len(job.df_fac) > 0:
        job.net_of_fac()    

        job.allocate_with_psold()   
        job.df_pat.to_csv(r'C:\_Working\PAT_20201019\__temp\df_pat_svc.csv',index=False)
    

# Run with Service

In [31]:
import requests
import uuid
import json

# files = {'para': ('pat_para', open('data/pat_job_test.json','rt'))}

job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid'] = str(uuid.uuid4())

files = {'para': ('pat_para', json.dumps(job_para))}
ret = requests.post("http://localhost:5000/api/Jobs", files = files)
ret.status_code, ret.content

(200, b'Analysis submitted!')

# Results Summary

In [23]:
import requests
import pandas as pd

ret = requests.get("http://localhost:5000/api/Jobs")

pd.DataFrame(ret.json())

,job_id,job_guid,job_name,receive_time,update_time,status,user_name,user_email
0,100000116,97bc4fcb-ec97-41f6-ad89-46691f7f7187 ...,PAT_Test_AIG,2021-09-30T23:28:42.294745,2021-09-30T23:28:42.294745,received,cxiao,chengyou.xiao@guycarp.com
1,100000115,ae8f1ece-b043-4366-bfa8-c44f3bdf06ef ...,PAT_Test_AIG,2021-09-30T23:24:21.783827,2021-09-30T23:24:21.783827,received,cxiao,chengyou.xiao@guycarp.com
2,100000114,d8e5f042-9192-4bdf-8de9-8059518c669c ...,PAT_Test_AIG,2021-09-30T23:21:04.473973,2021-09-30T23:22:52.118962,finished,cxiao,chengyou.xiao@guycarp.com
3,100000113,d0fde339-2b0d-4cce-9f4d-dd0fb37cb4f3 ...,PAT_Test_AIG,2021-09-30T23:01:22.724348,2021-09-30T23:02:22.853646,finished,cxiao,chengyou.xiao@guycarp.com
